In [132]:
import pandas as pd
import numpy as np
import glob as glob
import os
import time
import pathlib

# from ..route_imports import ratio_map
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
# import route_imports.ratio_map as ratios # store_load is a file on my library folder
from route_imports.ratio_map import metric_to_url_map # as ratios
from route_imports.ratio_map import url_to_metric_map
from route_imports.ratio_map import url_to_name_map
from route_imports.ratio_map import fin_statement_raw_names
from route_imports.ratio_map import fin_statement_renamed_cols
from route_imports.ratio_map import metric_to_formula_map
from route_imports.ratio_map import url_to_var_name_map
# from route_imports.ratio_map import metric_to_list_variables_map

# from route_imports.ratio_map import *

# from route_imports import ratio_map.metric_to_url_map# as ratios

start_time = time.time()
uri_list = "cash-and-cash-equivalents"
url_symbol_name = "apple"
url_symbol = "JAX"
url_fin_metric = "cash-and-cash-equivalents"

def magnitude_num(number, currency_symbol):
    if len(str(number)) > 9:
        magnitude = number/1000000000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"B")
    elif len(str(number)) > 6:
        magnitude = number/1000000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"M")
    elif len(str(number)) > 3:
        magnitude = number/1000
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"K")
    else:
        magnitude = number
        magnitude_str = "{}{}{}".format(currency_symbol,round(magnitude,1),"K")
    return magnitude_str

In [3]:
company_profiles = pd.read_csv("../reference_data/Company_Profiles.csv")#, encoding='cp1252')
fin_statements_matching = pd.read_csv("../reference_data/Financial_Statements_Reference_Matching.csv")#, encoding='cp1252')
currency_symbol = list(company_profiles[company_profiles['symbol']=="{}".format(url_symbol.upper())]['currency symbol'])[0]
company_profiles_col = ['symbol',
                        'long name',
                        'currency',
                        'exchange',
                        'industry',
                        'description',
                        'sector',
                        'country',
                        'ipo date',
                        'short name']
company_profiles = company_profiles[company_profiles_col]  

In [23]:
# hide - cash equiv
profiles_dict = {}
profiles_value = company_profiles[company_profiles['symbol']=="{}".format(url_symbol.upper())].values.tolist()[0]

titles_bs = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Balance Sheet"]['Title'])
titles_cf = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Cash Flow Statement"]['Title'])
titles_is = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Income Statement"]['Title'])

urls_bs = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Balance Sheet"]['URL'])
urls_cf = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Cash Flow Statement"]['URL'])
urls_is = list(fin_statements_matching[fin_statements_matching['Financial Statement']=="Income Statement"]['URL'])

In [24]:
# hide - cash equiv
for n, profiles_col in enumerate(company_profiles_col):
    key = profiles_col
    value = profiles_value[n]
    profiles_dict[key] = value 

if "{}".format(url_fin_metric) in urls_is:
    fin_statement_dir = "Income Statement"
elif "{}".format(url_fin_metric) in urls_bs:
    fin_metric_pos = urls_bs.index("{}".format(url_fin_metric))
    fin_statement_dir = "Balance Sheet"
    fin_statement_cols = titles_bs
    cols = titles_bs
    fin_metric_title = fin_statement_cols[fin_metric_pos]

elif "{}".format(url_fin_metric) in urls_cf:
    fin_statement_dir = "Cash Flow Statement"

else:
    pass


In [25]:
# hide - cash equiv
# start_time = time.time()
bs_dir = "Balance Sheet"
bs_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/quarter/{}/*~{}~*".format(bs_dir, url_symbol.upper()))[-1]

df = pd.read_csv(bs_file,index_col = False) #.format("NLOK"))[-1]
# df = df[df['date'].notna()]#fillna(method='ffill')
df = df[0:].iloc[::-1]#.dropna()
df_fin_statement = df
#region Pandas data manipulation
matching_row = fin_statements_matching[fin_statements_matching['URL']=="{}".format(url_fin_metric)]
fin_metric_title = list(matching_row['Title'])[0]
fin_metric_name = list(matching_row['Name'])[0]
print("fin metric name", fin_metric_name)
sorted_metric = df["{}".format(fin_metric_name)].sort_values()

fin metric name cashAndCashEquivalents


In [108]:
# fin_df_list[1]["period"]+"-"+fin_df_list[1]["date"].apply(lambda x: x[0:4])#["QQ-YYYY"]

In [12]:
fin_dir = ["Income Statement","Balance Sheet","Cash Flow Statement"]
# url_symbol = "TSLA"
url_symbol = "JAX"

fin_df_list = []
for x in fin_dir:
    fin_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/quarter/{}/*~{}~*".format(x, url_symbol.upper()))[-1]
    fin_df = pd.read_csv(fin_file)
    fin_df["Year"] = fin_df["date"].apply(lambda x: x[0:4])
    fin_df["QQ-YYYY"] = fin_df["period"]+"-"+fin_df["date"].apply(lambda x: x[0:4])#["QQ-YYYY"]
    fin_df_list.append(fin_df)
# df = pd.concat(fin_df_list,ignore_index=True)#,axis = 1)
# df = pd.concat(fin_df_list, join='outer', axis=1)#.fillna(nan_value)

# df = pd.concat(fin_df_list,ignore_index=True)
df_merge_is_bs = pd.merge(fin_df_list[0],fin_df_list[1],how="inner", on="QQ-YYYY")
# df
# df_merge cols
df_merge = pd.merge(df_merge_is_bs,fin_df_list[2],how="inner", on="QQ-YYYY")
df_merge = df_merge[fin_statement_raw_names]

df_merge.columns = fin_statement_renamed_cols
df = df_merge

df['ffo_math']=df['net_income'] + df['d_n_a'] + df['sales_maturities_of_investments'] + df['purchase_of_investments'] + df['investments_in_pp_n_e'] + df['acquisitions_net']
df['book_value_math']=df['total_assets'].dropna()-df['total_liabilities'].dropna()
df['ebit_math']=df['ebitda_non'] - df['d_n_a']
df['working_capital_math']= df['total_current_assets'] - df['total_current_liabilities']

df['quick_assets_math']=df['cash_non']+df['short_term_investments']+df['accounts_receivable']
df['quick_ratio_math']=df['total_current_assets'] - df['inventory']
# df['other_current_assets'] * df['total_current_assets']
# df['g_n_a']+df['s_n_m']

In [25]:
# wtf red dot
# perc​entage_of_debt_to_asset_formula = lambda perc​entage_of_debt_to_asset_formula: ((total_non_current_liabilities)/(total_assets))
# rece​ivables_turnover = lambda rece​ivables_turnover: ((net_revenue)/(accounts_receivable))

In [26]:
# # hide lambda
# net_working_capital_ratio = lambda net_working_capital_ratio: (df['working_capital_math']/df['total_assets'])
# book_value_per_share = lambda book_value_per_share: (df['total_se']/df['shares_outstanding_non'])
# total_equity_to_total_assets = lambda total_equity_to_total_assets: (df['total_se']/df['total_assets'])
# operating_cost_ratio = lambda operating_cost_ratio: (df['total_opex'] - df['d_n_a']/df['net_revenue'])
# percentage_of_debt_to_asset_formula = lambda percentage_of_debt_to_asset_formula: (df['total_non_current_liabilities']/df['total_assets'])

# total_liabilities_pct_of_total_assets = lambda total_liabilities_pct_of_total_assets: (df['total_liabilities']/df['total_assets'])
# debt_to_assets_ratio = lambda debt_to_assets_ratio: (df['total_debt']/df['total_assets'])
# debt_to_equity_ratio = lambda debt_to_equity_ratio: (df['total_debt']/df['total_se'])
# quick_ratio = lambda quick_ratio: (df['total_current_assets']-df['inventory']/df['total_current_liabilities'])
# working_capital = lambda working_capital: (df['total_current_assets'] - df['total_current_liabilities'])
# current_ratio = lambda current_ratio: (df['total_current_assets']/df['total_current_liabilities'])
# capital_intensity = lambda capital_intensity: (df['total_assets']/df['net_revenue'])
# equity_multiplier = lambda equity_multiplier: (df['total_assets']/df['total_se'])
# short_term_debt_to_equity_ratio = lambda short_term_debt_to_equity_ratio: (df['short_term_debt']/df['total_se'])
# st_debt_as_pct_of_total_debt = lambda st_debt_as_pct_of_total_debt: (df['short_term_debt']/df['total_liabilities'])
# acid_test_ratio = lambda acid_test_ratio: (df['quick_assets_math']/df['total_current_liabilities'])
# pre_tax_income_to_sales = lambda pre_tax_income_to_sales: (df['pretax_income_non']/df['net_revenue'])
# pre_tax_return_on_assets = lambda pre_tax_return_on_assets: (df['pretax_income_non']/df['total_assets'])
# pre_tax_return_on_common_equity = lambda pre_tax_return_on_common_equity: (df['pretax_income_non']/df['common_stock'])
# operating_roa = lambda operating_roa: (df['operating_income']/df['total_assets'])
# operating_profit_margin = lambda operating_profit_margin: (df['operating_income']/df['net_revenue'])
# free_operating_cash_flow_to_debt = lambda free_operating_cash_flow_to_debt: (df['operating_cash_flow']-df['capex']/df['total_debt'])
# discretionary_cash_flow_to_debt = lambda discretionary_cash_flow_to_debt: (df['operating_cash_flow']-df['accounts_payable']-df['debt_repayment']-df['dividends_paid']-df['interest_expense'])
# operating_cash_flow_to_interest = lambda operating_cash_flow_to_interest: (df['operating_cash_flow'] + df['interest_expense'] + df['income_tax_expense']/df['interest_expense'])
# operating_cash_flow_to_debt = lambda operating_cash_flow_to_debt: (df['operating_cash_flow'] + df['interest_expense'] + df['income_tax_expense']/df['total_liabilities'])
# cash_flow_margin_ratio_formula = lambda cash_flow_margin_ratio_formula: (df['operating_cash_flow']/df['total_liabilities'])
# cash_flow_to_debt = lambda cash_flow_to_debt: (df['operating_cash_flow']/df['total_debt'])
# net_cash_flow_to_capital_expenditures = lambda net_cash_flow_to_capital_expenditures: (df['operating_cash_flow']/df['capex'])
# cash_flow_to_revenue = lambda cash_flow_to_revenue: (df['operating_cash_flow']/df['net_revenue'])
# cash_return_on_assets = lambda cash_return_on_assets: (df['operating_cash_flow']/df['total_assets'])
# cash_return_on_equity = lambda cash_return_on_equity: (df['operating_cash_flow']/df['total_se'])
# cash_to_income_ratio = lambda cash_to_income_ratio: (df['operating_cash_flow']/df['operating_income'])
# cash_flow_per_share = lambda cash_flow_per_share: (df['operating_cash_flow']/df['shares_outstanding_non'])
# debt_payment = lambda debt_payment: (df['operating_cash_flow']/df['debt_repayment'])
# debt_coverage = lambda debt_coverage: (df['operating_cash_flow']/df['long_term_debt'])
# cash_flow_from_operations_ratio = lambda cash_flow_from_operations_ratio: (df['operating_cash_flow']/df['total_current_liabilities'])
# gross_profit_margin = lambda gross_profit_margin: (df['net_revenue'] - df['cost_of_sales']/df['net_revenue'])
# receivables_turnover = lambda receivables_turnover: (df['net_revenue']/df['accounts_receivable'])
# capital_turnover_ratio = lambda capital_turnover_ratio: (df['net_revenue']/df['working_capital_math'])
# assets_turnover_ratio = lambda assets_turnover_ratio: (df['net_revenue']/df['total_assets'])
# accounts_receivableturnover = lambda accounts_receivableturnover: (df['net_revenue']/df['accounts_receivable'])
# operating_cash_flow_to_debt = lambda operating_cash_flow_to_debt: (df['net_revenue']/df['total_se'])
# inventory_ratio = lambda inventory_ratio: (df['net_revenue']/df['inventory'])
# return_on_investment = lambda return_on_investment: (df['net_income']+ df['interest_expense']/df['total_se'])
# pretax_margin = lambda pretax_margin: (df['net_income'] - df['income_tax_expense']/df['net_revenue'])
# income_to_net_worth_ratio = lambda income_to_net_worth_ratio: (df['net_income'] - df['deferred_income_tax']/df['shares_outstanding_non'])
# return_on_assets_roa = lambda return_on_assets_roa: (df['net_income']/df['total_assets'])
# roe = lambda roe: (df['net_income']/df['total_se'])
# profit_margin = lambda profit_margin: (df['net_income']/df['net_revenue'])
# earnings_per_share = lambda earnings_per_share: (df['net_income']/df['shares_outstanding_non'])
# current_cash_debt_coverage = lambda current_cash_debt_coverage: (df['net_cash_by_operating_activities']/df['total_current_liabilities'])
# cash_debt_coverage = lambda cash_debt_coverage: (df['net_cash_by_operating_activities']/df['total_liabilities'])
# long_term_debt_ratio = lambda long_term_debt_ratio: (df['long_term_debt']/df['total_assets'])
# long_term_debt_equity_ratio = lambda long_term_debt_equity_ratio: (df['long_term_debt']/df['total_se'])
# lt_debt_as_pct_of_total_debt = lambda lt_debt_as_pct_of_total_debt: (df['long_term_debt']/df['total_liabilities'])
# inventory_pct_of_revenue = lambda inventory_pct_of_revenue: (df['inventory']/df['net_revenue'])
# intangibles_pct_of_book_value = lambda intangibles_pct_of_book_value: (df['goodwill_n_intangible_assets']/df['total_se'])
# ffo_funds_from_operations_to_debt = lambda ffo_funds_from_operations_to_debt: (df['ffo_math']/df['total_debt'])
# operating_margin = lambda operating_margin: (df['ebitda_non']-df['d_a']/df['net_revenue'])
# cash_coverage_ratio = lambda cash_coverage_ratio: (df['ebitda_non']/df['interest_expense'])
# ebitda_per_share = lambda ebitda_per_share: (df['ebitda_non']/df['shares_outstanding_non'])
# ebitda_interest_coverage = lambda ebitda_interest_coverage: (df['ebitda_non']/df['interest_expense'])
# ebitda_margin = lambda ebitda_margin: (df['ebitda_non']/df['net_revenue'])
# net_margin_profit_margin = lambda net_margin_profit_margin: (df['ebitda_margin']/df['net_revenue'])
# return_on_capital_employed_ratio = lambda return_on_capital_employed_ratio: (df['ebit_math']/df['total_assets'] - df['total_current_liabilities'])
# debt_service_ratio = lambda debt_service_ratio: (df['ebit_math']/df['interest_expense'])
# return_on_capital = lambda return_on_capital: (df['ebit_math']/df['total_assets'])
# dividend_yield = lambda dividend_yield: (df['dividends_paid']/df['shares_outstanding_non'])
# dividend_payout_ratio = lambda dividend_payout_ratio: (df['dividends_paid']/df['net_income'])
# dividends_per_share = lambda dividends_per_share: (df['dividends_paid']/df['shares_outstanding_non'])
# inventory_turnover = lambda inventory_turnover: (df['cost_of_sales']/df['inventory'])
# cash_ratio = lambda cash_ratio: (df['cash_non']/df['total_current_liabilities'])
# cash_flow_roa = lambda cash_flow_roa: (df['operating_cash_flow']/df['total_assets'])
# average_collection_period = lambda average_collection_period: (df['accounts_receivable']/df['net_revenue'])
# number_of_days_of_receivables = lambda number_of_days_of_receivables: (df['accounts_receivable']/df['inventory'])
# average_days_payables_outstanding = lambda average_days_payables_outstanding: (df['accounts_payable']/df['cost_of_sales'])
# days_sales_in_payables = lambda days_sales_in_payables: (df['accounts_payable']/df['total_opex'])

In [27]:
# ratios_df = pd.read_csv("../reference_data/ratios.csv")
# ratios_df = ratios_df.dropna(subset = ["Name"])
# fin_statements_matching[fin_statements_matching['URL']=="{}".format(url_fin_metric)]
# fin_metric_ratio_variable_name = list(matching_row['Python Variable Name'])[0]
# # ratios_df = pd.read_csv("../reference_data/ratios.csv")

In [15]:
df['year'] = df['filing_date'].apply(lambda x: str(x)[0:4])
year_df = df.groupby("year").sum()

In [16]:
year_df = df.groupby("year").sum()

In [107]:
# metric_name = url_to_metric_map['common-stock-repurchased']#['net-working-capital-ratio']
# metric_to_formula_map(df,metric_name)#.items()

In [32]:
url_to_metric_map['net-working-capital-ratio']

'net_working_capital_ratio'

In [33]:
# net_working_capital_ratio[url_to_metric_map['net-working-capital-ratio']]

In [21]:
# url_fin_metric = 'cash-and-cash-equivalents'
url_fin_metric = 'net-working-capital-ratio'
# fin_statements_matching = pd.read_csv("../reference_data/Financial_Statements_Reference_Matching.csv")#, encoding='cp1252')

In [24]:
# url_fin_metric = 'net-working-capital-ratio'

fin_metric_title = url_to_name_map[url_fin_metric]
fin_metric_name = url_fin_metric
metric_history = metric_to_formula_map(df,metric_name)
sorted_metric = metric_history.sort_values(ascending=True)

In [25]:
df['metric history'] = metric_history

In [29]:
df['metric history'] = metric_history
df['period'] = df['filing_date'].apply(lambda x: str(x)[0:4])
year_df = df.groupby("year").sum()

In [210]:
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Balance Sheet/quarter~JCP~balance_sheet.csv")
df = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Income Statement/quarter~JCP~income_statement.csv")
year_df = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/year/Income Statement/year~JCP~income_statement.csv")
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Cash Flow Statement/quarter~JCP~cash_flow_statement.csv")
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Technology/quarter/Income Statement/quarter~AAPL~income_statement.csv")
# jcp_bs = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Consumer Cyclical/quarter/Cash Flow Statement/quarter~JAX~cash_flow_statement.csv")
# jcp_bs
# df = pd.read_csv("D:/Cloud/rclone/OneDrive/Web/TenDollarData/Charts_TenDollarData/financial_statements/data/Historical Financial Statements/Technology/quarter/Cash Flow Statement/quarter~AAPL~cash_flow_statement.csv")

In [216]:
pd.to_datetime(year_df.index.apply(lambda x: x[0:4])).values.astype(np.int64) // 10 ** 6

array([1577836800000, 1577836800000, 1546300800000, 1546300800000,
       1546300800000, 1546300800000, 1514764800000, 1514764800000,
       1514764800000, 1514764800000, 1483228800000, 1483228800000,
       1483228800000, 1483228800000, 1451606400000, 1451606400000,
       1451606400000, 1451606400000, 1420070400000, 1420070400000,
       1420070400000, 1420070400000, 1388534400000, 1388534400000,
       1388534400000, 1388534400000, 1356998400000, 1356998400000,
       1356998400000, 1356998400000, 1325376000000, 1325376000000,
       1325376000000, 1325376000000, 1293840000000, 1293840000000,
       1293840000000, 1293840000000, 1262304000000, 1262304000000,
       1262304000000, 1262304000000, 1230768000000, 1230768000000,
       1230768000000, 1230768000000, 1199145600000, 1199145600000,
       1199145600000, 1199145600000, 1167609600000, 1167609600000,
       1167609600000, 1167609600000, 1167609600000, 1136073600000,
       1136073600000, 1136073600000, 1136073600000, 1104537600

In [171]:
metric_to_list_variables_map = {
        'net_working_capital_ratio':['working_capital_math','total_assets',],
        'book_value_per_share':['total_se','shares_outstanding_non',],
        'total_equity_to_total_assets':['total_se','total_assets',],
        'operating_cost_ratio':['total_opex','d_n_a','net_revenue',],
        'percentage_of_debt_to_asset_formula':['total_non_current_liabilities','total_assets',],

        'total_liabilities_pct_of_total_assets':['total_liabilities','total_assets',],
        'debt_to_assets_ratio':['total_debt','total_assets',],
        'debt_to_equity_ratio':['total_debt','total_se',],
        'quick_ratio':['total_current_liabilities','total_assets'],
        'working_capital':['total_current_assets','total_current_liabilities',],
        'current_ratio':['total_current_liabilities','total_assets','inventory'],
        'capital_intensity':['total_assets','net_revenue',],
        'equity_multiplier':['total_assets','total_se',],
        'short_term_debt_to_equity_ratio':['short_term_debt','total_se',],
        'st_debt_as_pct_of_total_debt':['short_term_debt','total_liabilities',],
        'acid_test_ratio':['quick_assets_math','total_current_liabilities',],
        'pre_tax_income_to_sales':['pretax_income_non','net_revenue',],
        'pre_tax_return_on_assets':['pretax_income_non','total_assets',],
        'pre_tax_return_on_common_equity':['pretax_income_non','common_equity',],
        'operating_roa':['operating_income','total_assets',],
        'operating_profit_margin':['operating_income','net_revenue',],
        'free_operating_cash_flow_to_debt':['operating_cash_flow','capex','total_debt',],
        'discretionary_cash_flow_to_debt':['operating_cash_flow','accounts_payable','debt_repayment','dividends_paid','interest_expense',],
        'operating_cash_flow_to_interest':['operating_cash_flow','interest_expense','income_tax_expense','interest_expense',],
        'operating_cash_flow_to_debt':['operating_cash_flow','interest_expense','income_tax_expense','total_liabilities',],
        'cash_flow_margin_ratio_formula':['operating_cash_flow','total_liabilities',],
        'cash_flow_to_debt':['operating_cash_flow','total_debt',],
        'net_cash_flow_to_capital_expenditures':['operating_cash_flow','capex',],
        'cash_flow_to_revenue':['operating_cash_flow','net_revenue',],
        'cash_return_on_assets':['operating_cash_flow','total_assets',],
        'cash_return_on_equity':['operating_cash_flow','total_se',],
        'cash_to_income_ratio':['operating_cash_flow','operating_income',],
        'cash_flow_per_share':['operating_cash_flow','shares_outstanding_non',],
        'debt_payment':['operating_cash_flow','debt_repayment',],
        'debt_coverage':['operating_cash_flow','long_term_debt',],
        'cash_flow_from_operations_ratio':['operating_cash_flow','total_current_liabilities',],
        'gross_profit_margin':['net_revenue','cost_of_sales','net_revenue',],
        'rece​ivables_turnover':['net_revenue','accounts_receivable',],
        'capital_turnover_ratio':['net_revenue','working_capital_math',],
        'assets_turnover_ratio':['net_revenue','total_assets',],
        'accounts_receivableturnover':['net_revenue','accounts_receivable',],
        'operating_cash_flow_to_debt':['net_revenue','total_se',],
        'inventory_ratio':['net_revenue','inventory',],
        'return_on_investment':['net_income','interest_expense','total_se','long_term_debt',],

        'pretax_margin':['net_income','income_tax_expense','net_revenue',],
        'income_to_net_worth_ratio':['net_income','deferred_income_tax','shares_outstanding_non',],
        'return_on_assets_roa':['net_income','total_assets',],
        'roe':['net_income','total_se',],
        'profit_margin':['net_income','net_revenue',],
        'earnings_per_share':['net_income','shares_outstanding_non',],
        'current_cash_debt_coverage':['net_cash_by_operating_activities','total_current_liabilities',],
        'cash_debt_coverage':['net_cash_by_operating_activities','total_liabilities',],
        'long_term_debt_ratio':['long_term_debt','total_assets',],
        'long_term_debt_equity_ratio':['long_term_debt','total_se',],
        'lt_debt_as_pct_of_total_debt':['long_term_debt','total_liabilities',],
        'inventory_pct_of_revenue':['inventory','net_revenue',],
        'intangibles_pct_of_book_value':['goodwill_n_intangible_assets','total_se',],
        'ffo_funds_from_operations_to_debt':['ffo_math','total_debt',],
        'operating_margin':['ebitda_non','d_a','net_revenue',],
        'cash_coverage_ratio':['ebitda_non','interest_expense',],
        'ebitda_per_share':['ebitda_non','shares_outstanding_non',],
        'ebitda_interest_coverage':['ebitda_non','interest_expense',],
        'ebitda_margin':['ebitda_non','net_revenue',],
        'net_margin_profit_margin':['ebitda_margin','net_revenue',],
        'return_on_capital_employed_ratio':['ebit_math','total_assets','total_current_liabilities',],
        'debt_service_ratio':['ebit_math','interest_expense',],
        'return_on_capital':['ebit_math','total_assets',],
        'dividend_yield':['dividends_paid','price_market','shares_outstanding_non',],
        'dividend_payout_ratio':['dividends_paid','net_income',],
        'dividends_per_share':['dividends_paid','shares_outstanding_non',],
        'inventory_turnover':['cost_of_sales','inventory',],
        'cash_ratio':['cash_non','total_current_liabilities',],
        'cash_flow_roa':['operating_cash_flow','total_assets',],
        'average_collection_period':['accounts_receivable','net_revenue',],
        'number_of_days_of_receivables':['accounts_receivable','inventory',],
        'average_days_payables_outstanding':['accounts_payable','cost_of_sales',],
        'days_sales_in_payables':['accounts_payable','total_opex',],
    }

In [172]:
fin_dir = ["Income Statement","Balance Sheet","Cash Flow Statement"]
# url_symbol = "TSLA"
url_symbol = "JCP" # "AAPL"

# url_fin_metric = 'current-ratio'
url_fin_metric = "operating-cost-ratio" # 'discretionary-cash-flow-to-debt'
fin_df_list = []
for x in fin_dir:
    fin_file = glob.glob("../Charts_TenDollarData/financial_statements/data/Historical Financial Statements/*/quarter/{}/*~{}~*".format(x, url_symbol.upper()))[-1]
    fin_df = pd.read_csv(fin_file)
    fin_df["Year"] = fin_df["date"].apply(lambda x: x[0:4])
    fin_df["QQ-YYYY"] = fin_df["period"]+"-"+fin_df["date"].apply(lambda x: x[0:4])#["QQ-YYYY"]
    fin_df_list.append(fin_df)
# df = pd.concat(fin_df_list,ignore_index=True)#,axis = 1)
# df = pd.concat(fin_df_list, join='outer', axis=1)#.fillna(nan_value)

# df = pd.concat(fin_df_list,ignore_index=True)
df_merge_is_bs = pd.merge(fin_df_list[0],fin_df_list[1],how="inner", on="QQ-YYYY")
df_merge = pd.merge(df_merge_is_bs,fin_df_list[2],how="inner", on="QQ-YYYY")
df_merge = df_merge[fin_statement_raw_names]

df_merge.columns = fin_statement_renamed_cols
df = df_merge
metric_name = url_to_metric_map["{}".format(url_fin_metric)]
fin_metric_title = url_to_name_map[url_fin_metric] 
fin_metric_name = url_to_var_name_map[url_fin_metric]
fin_metric_vars = metric_to_list_variables_map[fin_metric_name]

df['ffo_math']=df['net_income'] + df['d_n_a'] + df['sales_maturities_of_investments'] + df['purchase_of_investments'] + df['investments_in_pp_n_e'] + df['acquisitions_net']
df['book_value_math']=df['total_assets'].dropna()-df['total_liabilities'].dropna()
df['ebit_math']=df['ebitda_non'] - df['d_n_a']
df['working_capital_math']= df['total_current_assets'] - df['total_current_liabilities']

df['quick_assets_math']=df['cash_non']+df['short_term_investments']+df['accounts_receivable']
df['quick_ratio_math']=df['total_current_assets'] - df['inventory']
# df['other_current_assets'] * df['total_current_assets']
# df['g_n_a']+df['s_n_m']

In [92]:
if len(df) == len(df[df['revenue'] == 0]):
    return 404

18

In [96]:
# # hide
# filter_pos_neg = list(fin_statements_matching[fin_statements_matching['Name']=="revenue"]['positive_negative'])[0]
# filter_pos_neg_list = ['positive','all', 'both', 'negative', 'zero_or_neg']


# if "{}".format(filter_pos_neg) == "positive": #in filter_pos_neg_list:
#     nan_rows = df[(df['revenue']<0)]
#     nan_rows['revenue'] = np.nan
#     df[(df['revenue']<0)] = nan_rows
# #     df.loc[df['revenue'] < 0, 0:] = np.nan
# #     df.loc[df['revenue'] < 0, 0:] = np.nan
# #     df = df[~(df['revenue']<0)]
# elif "{}".format(filter_pos_neg) == "neg": #in filter_pos_neg_list:
#     nan_rows = df[(df['revenue']>=0)]
#     nan_rows['revenue'] = np.nan
#     df[(df['revenue']>=0)] = nan_rows
# #     df.loc[df['revenue'] > 0, 0:] = np.nan
# #     df = df[~(df['revenue']>=0)]
# elif "{}".format(filter_pos_neg) == "zero_or_neg":
#     nan_rows = df[(df['revenue']>0)]
#     nan_rows['revenue'] = np.nan
#     df[(df['revenue']>0)] = nan_rows
# #     df.loc[df['revenue'] > 0, 0:] = np.nan
# #     df = df[~(df['revenue']>0)]
# elif "{}".format(filter_pos_neg) == "both":
#     nan_rows = df[(df['revenue']==0)]
#     nan_rows['revenue'] = np.nan
#     df[(df['revenue']==0)] = nan_rows
# #     df.loc[df['revenue'] == 0, 0:] = np.nan
# #     df = df[~(df['revenue']==0)]
# elif "{}".format(filter_pos_neg) == "all":
#     pass
# else:
#     pass
# df['revenue'] = df['revenue'].fillna(df['revenue'].rolling(4,center=True,min_periods=1).mean())
# # df

In [165]:
fin_metric_varsz

['total_current_liabilities', 'total_assets', 'inventory']

In [173]:
filter_pos_neg_list = ['positive','all', 'both', 'negative', 'zero_or_neg']
for n,x in enumerate(fin_metric_vars):
    filter_pos_neg = list(fin_statements_matching[fin_statements_matching['Python Variable Name']=="{}".format(x)]['positive_negative'])[0]
    if "{}".format(filter_pos_neg) == "positive": #in filter_pos_neg_list:
        nan_rows = df[(df["{}".format(x)]<0)]
        nan_rows['{}'.format(x)] = np.nan
        df[(df['{}'.format(x)]<0)] = nan_rows
    elif "{}".format(filter_pos_neg) == "neg": #in filter_pos_neg_list:
        nan_rows = df[(df['{}'.format(x)]>=0)]
        nan_rows['{}'.format(x)] = np.nan
        df[(df['{}'.format(x)]>=0)] = nan_rows
    elif "{}".format(filter_pos_neg) == "zero_or_neg":
        nan_rows = df[(df['{}'.format(x)]>0)]
        nan_rows['{}'.format(x)] = np.nan
        df[(df['{}'.format(x)]>0)] = nan_rows
    elif "{}".format(filter_pos_neg) == "both":
        nan_rows = df[(df['{}'.format(x)]==0)]
        nan_rows['{}'.format(x)] = np.nan
        df[(df['{}'.format(x)]==0)] = nan_rows
    elif "{}".format(filter_pos_neg) == "all":
        pass
    else:
        pass
#     df['revenue'] = df['revenue'].fillna(df['revenue'].rolling(4,center=True,min_periods=1).mean())
    # df

<ipython-input-173-44e185a7fd55>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_rows['{}'.format(x)] = np.nan
<ipython-input-173-44e185a7fd55>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_rows['{}'.format(x)] = np.nan
<ipython-input-173-44e185a7fd55>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [181]:
df['{}'.format(x)] = df['{}'.format(x)].fillna(df['{}'.format(x)].rolling(1000,center=True,min_periods=1).mean())

In [198]:
pct_chg = df['net_revenue']/df['net_revenue'].shift(-4)
pct_chg = pct_change.apply(lambda x: "{}%".format(round((x-1)*100,1)))

In [187]:
df['net_revenue'][::-1].pct_change(periods=4)

79         NaN
78         NaN
77         NaN
76         NaN
75   -0.519798
        ...   
4    -0.043429
3    -0.074231
2    -0.085254
1    -0.077390
0     0.618153
Name: net_revenue, Length: 80, dtype: float64

In [ ]:
df['revenue'] = df['revenue'].fillna(df['revenue'].rolling(4,center=True,min_periods=1).mean())

if df['revenue'][0]>=0 and (len(df['revenue'])) - df['revenue'].lt(0).sum() == 1:
    df.loc[df['revenue'] > 0, 0:] = np.nan
#     jcp_bs.replace()
elif df['revenue'][0]<=0 and (len(df['revenue'])) - df['revenue'].gt(0).sum()==1:
    df.loc[df['revenue']<0, 'revenue'] = np.nan
else:
    print("passed")

In [49]:
fin_statements_matching['positive_negative'].unique()

from collections import Counter

words = list(fin_statements_matching['positive_negative'])#[0]

print(Counter(words).keys())
print(Counter(words).values()) # counts the elements' frequency

dict_keys(['-', 'positive', 'all', 'both', 'negative', 'zero_or_neg', 'zero_or_pos', nan])
dict_values([17, 46, 15, 27, 2, 6, 1, 2])


In [18]:
# important
# jcp_bs['otherFinancingActivities'][::-1].values.searchsorted(0, side='right')
# list(jcp_bs['costAndExpenses'].pct_change())
# list(jcp_bs['costAndExpenses'])

In [50]:
early_missing_periods = jcp_bs[::-1]['commonStockRepurchased'].ne(0).idxmax()
jcp_bs = jcp_bs[0:early_missing_periods+1]

In [53]:
# jcp_bs.loc[jcp_bs.commonStockRepurchased>0].iloc[[0,]].index[0]

In [52]:
# jcp_bs.loc[(jcp_bs.commonStockRepurchased>3).groupby(jcp_bs.symbol).idxmax]

In [48]:

ignore_periods = jcp_bs['commonStockRepurchased'][::-1].values.searchsorted(0, side='right')
df = jcp_bs[0:ignore_periods]

In [150]:
# jcp_bs['revenue']= 
# jcp_bs['revenue'].fillna(pd.rolling_mean(input_data_frame[var_list], 6, min_periods=1))
# jcp_bs['revenue'].fillna(pd.rolling(4,center=True,min_periods=1).mean((input_data_frame[var_list], 6, min_periods=1)))
# jcp_bs['revenue'].fillna(jcp_bs['revenue'].rolling(4, min_periods=1, center=True))
jcp_bs['revenue'] = jcp_bs['revenue'].fillna(jcp_bs['revenue'].rolling(4,center=True,min_periods=1).mean())
jcp_bs['revenue'] = jcp_bs['revenue'].replace(np.NaN, 0)

In [68]:
no_null_jcp = jcp_bs[~jcp_bs['retainedEarnings'].isnull()]
no_neg_jcp = no_null_jcp[~(no_null_jcp['retainedEarnings']<0)]
no_pos_jcp = no_neg_jcp[~(no_neg_jcp['retainedEarnings']>=0)]

In [116]:
# important
jcp_bs[(jcp_bs['otherFinancingActivites'].isnull())&(jcp_bs['otherFinancingActivites']<0)]
# jcp_bs[~(jcp_bs['retainedEarnings']<0)]

,Unnamed: 0,date,symbol,fillingDate,acceptedDate,period,netIncome,depreciationAndAmortization,deferredIncomeTax,stockBasedCompensation,...,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink


In [79]:
if jcp_bs['revenue'][0]>=0 and (len(jcp_bs['revenue'])) - jcp_bs['revenue'].lt(0).sum() == 1:
    jcp_bs.loc[jcp_bs['revenue'] > 0, 0:] = np.nan
#     jcp_bs.replace()
elif jcp_bs['revenue'][0]<=0 and (len(jcp_bs['revenue'])) - jcp_bs['revenue'].gt(0).sum()==1:
    jcp_bs.loc[jcp_bs['revenue']<0, 'revenue'] = np.nan
else:
    print("passed")

In [155]:
jcp_bs[['totalOtherIncomeExpensesNet']].to_html().replace("<td>NaN</td>","<td></td>")

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>totalOtherIncomeExpensesNet</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td></td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>-4000000.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>13000000.0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>13000000.0</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>13000000.0</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>14000000.0</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>19000000.0</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>19000000.0</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>19000000.0</td>\n    </tr>\n    <tr>\n      <th>9</th>\n      <td>-8000000.0</td>\n    </tr>\n    <tr>\n      <th>10</th>\n      <td>2000000.0</td>\n    </tr>\n    <tr>\n      <th>11</th>\n      <td>14000000.0</td>\n    </tr>\n    <tr>\n      <th>12</th>\n      

In [8]:
# sorted_metric = np.sort(metric_history)[::-1]
fin_metric_name = 'cashAndCashEquivalents'
sorted_metric = df["{}".format(fin_metric_name)].sort_values()
quarters = round(len(sorted_metric)/4)
bottom_25 = sorted_metric[len(sorted_metric)-1-quarters]
top_25 = sorted_metric[quarters-1]
max_num = max(sorted_metric)
min_num = min(sorted_metric)
mean = sorted_metric.mean()
std_dev = sorted_metric.std()
std_dev_str = "+/-{}{}%".format(currency_symbol,round(abs((std_dev-mean)/mean)*100,1))
mean_str = magnitude_num(mean, currency_symbol)
max_str = magnitude_num(max_num, currency_symbol)
min_str = magnitude_num(min_num, currency_symbol)
bottom_25_str = magnitude_num(bottom_25, currency_symbol)
top_25_str = magnitude_num(top_25, currency_symbol)

In [9]:
latest_num = df["{}".format(fin_metric_name)][0]
first_num = list(df["{}".format(fin_metric_name)])[-1]
pct_chg = (latest_num - first_num)/first_num
historical_pct_chg = str(round(pct_chg*100, 1))
# lifetime_sum_all_metric = df["{}".format(fin_metric_name)].sum()
# lifetime_sum_all_metric = magnitude_num(lifetime_sum_all_metric,currency_symbol)

In [11]:
# hide - cash equiv
if pct_chg>=0:
    pct_chg_str = "+{}%".format(pct_chg)
elif pct_chg<0:
    pct_chg_str = "-{}%".format(pct_chg)
else:
    pct_chg_str = ""
# historical_pct_chg = str(round(pct_chg*100, 1))
max_min_pct_diff = ((max_num-min_num)/min_num)
if max_min_pct_diff>=0:
    max_min_pct_diff_str = "+{}%".format(round(max_min_pct_diff)*100,1)
elif max_min_pct_diff<0:
    max_min_pct_diff_str = "-{}%".format(round(max_min_pct_diff)*100,1)
else:
    max_min_pct_diff_str = ""
    
df_fin_statement['Quarter & Year'] =(df_fin_statement['date'].astype(str).str[0:4]).astype(int)
latest_year = list((df_fin_statement['date'].astype(str).str[0:4]))[0]
earliest_year = list((df_fin_statement['date'].astype(str).str[0:4]))[-1]
earliest_metric = (list(df["{}".format("{}".format(fin_metric_name))])[-1])
latest_metric = (list(df["{}".format("{}".format(fin_metric_name))])[0])

In [13]:
fin_metric_title

'Cash and Cash Equivalents'

In [12]:
titles_bs =['Cash and Cash Equivalents',
 'Short Term Investments',
 'Cash And Short-Term Investments',
 'Accounts Receivables',
 'Inventory',
 'Other Current Assets',
 'Total Current Assets',
 'PP&E',
 'Goodwill',
 'Intangible Assets',
 'Goodwill and Intangible Assets',
 'Long-term Investments',
 'Tax Assets',
 'Other Non-Current Assets',
 'Total Non-Current Assets',
 'Other Assets',
 'Total Assets',
 'Accounts Payables',
 'Short-term Debt',
 'Income Tax Payables',
 'Deferred Revenue',
 'Other Current Liabilities',
 'Total Current Liabilities',
 'Long-term Debt',
 'Deferred Revenue Non-Current',
 'Deferred Tax Liabilities Non-Current',
 'Other Non-Current Liabilities',
 'Total Non-Current Liabilities',
 'Other Liabilities',
 'Total Liabilities',
 'Common Stock',
 'Retained Earnings',
 'Accumulated Other Comprehensive Income Loss',
 'Other Shareholders Equity',
 'Total Shareholders Equity',
 'Total Liabilities & Stockholders Equity',
 'Total Investments',
 'Total Debt',
 'Net Debt',
 'SEC Statement Link']
# col = titles_bs
# df_fin_statement = df_fin_statement[col]
# df_fin_statement = df_fin_statement.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]]
# cols

In [13]:
# hide - cash equiv
titles_list = ['Date','Symbol','Filing Date','Accepted Date','Period','SEC Filing Link'][:]

for x in reversed(titles_bs):
    if x in titles_list:
        titles_bs.remove(x)
        print(x, "is in")
    else:
        pass
#         print(x, "is not in")  
titles_bs.append('Quarter & Year') 
df_fin_statement = df_fin_statement.drop([ 'Unnamed: 0','date','symbol','fillingDate','acceptedDate','period','link'],axis=1, errors='ignore')
df_fin_statement.columns = titles_bs

# df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_title = "Cash and Cash Equivalents"
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)

elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

In [15]:
# hide - variables
# vars_names_list = []
# vars_values_list = []
# # for name in vars().keys():
# #   vars_names_list.append(name)
# for name, value in globals().items():
#     vars_names_list.append(name)
#     vars_values_list.append(value)
# vars_names_list    
# vars_values_list_100 = [str(x)[0:50] for x in vars_values_list]
# vars_df = pd.DataFrame({'vars_names_list':vars_names_list,'vars_values_list_100':vars_values_list_100})
# # vars_values_list_100
# vars_df.to_csv("../reference_data/flask_variables.csv")

In [14]:
df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]
# pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
# df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
# df_pct_chg_t = df_pct_chg_t.iloc[1:]
# # list(df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)])
# # len(df_pct_chg.transpose())
# # list(df_pct_chg['Quarter & Year'])
# df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

In [100]:
# hide
df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
# df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]
df_pct_chg = df_fin_statement

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
# df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
# df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
# df_pct_chg_t = df_pct_chg_t.iloc[1:]
# df_pct_chg_t
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

# print("ww",len(col))
cols = list(df_pct_chg_t.columns)
print("ww",len(df))
cols = [cols[-1]] + cols[:-1]

df_pct_chg_t = df_pct_chg_t[cols]

df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index

df_t.index = range(len(df_t))
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
df_pct = df_pct_chg_t.to_html()#.replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

In [39]:

df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]

df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())

new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum#[1:] #take the data less the header row

df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
# df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
# df_t = df_t[0:25]

df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
len(list(df_t))
df_t.columns
df_t[df_t.columns]

Index([1085.22,      '',    1994,    1994,    1994,    1994,    1994,    1994,
          1994,    1994,
       ...
          2019,    2020,    2020,    2020,    2020,    2020,    2020,    2020,
          2020,    2020],
      dtype='object', length=429)

In [19]:

# df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])


Nothing took 19.73895835876465 seconds


#### trying to read flask server with requests  

In [13]:
import requests
response = requests.get("http://www.google.com")
response = requests.get("http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents")
print(response.content)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>NameError: name \'start_time\' is not defined // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script type="text/javascript">\n      var TRACEBACK = 1834011132928,\n          CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "t3wb8w77FxDSu3AUMLC3";\n    </

In [14]:
import urllib.request

myurl = "http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents"

# # GET is the default action
# response = urllib.request.urlopen(myurl)  

# # Output from the GET assuming response code was 200
# data = response.read()  

with urllib.request.urlopen("http://127.0.0.1:5000/aapl-stock/apple/cash-and-cash-equivalents") as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    print(s)

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [ ]:
from requests_testadapter import Resp

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
requests_session.get('file://<some_local_path>')

In [15]:
import requests

os.environ['NO_PROXY'] = '127.0.0.1'
r = requests.get('http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents')
print(r.content)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'


In [99]:
import requests
os.environ['NO_PROXY'] = '127.0.1'

# r=requests.get("http://127.0.01:5000/aapl=stock/apple/cash-and-cash-equivalents")

In [16]:
# BASE_URL='http://127.0.0.1:8000'
BASE_URL = "http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents/"
BASE_URL = "http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents"
resp=requests.get(BASE_URL)
# print(resp.status_code)
# print(resp.json())

In [17]:
resp.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [19]:
req = requests.post("http://127.0.0.1:5000/aapl=stock/apple/cash-and-cash-equivalents", 'html.parser')
req

<Response [404]>

In [98]:
from pypac import PACSession
session = PACSession()
# session.get('https://127.0.0.1/aapl=stock/apple/cash-and-cash-equival
# session.get("http://127.0.01:5000/aapl=stock/apple/cash-and-cash-equivalents")
# <Response [200]>

In [ ]:
from pypac import PACSession
session = PACSession()
session.get('https://127.0.0/aapl=stock/apple/cash-and-cash-equivalents')
# <Response [200]>

In [ ]:
from requests_testadapter import Resp

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
requests_session.get('file://<some_local_path>')

In [17]:








# df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)


elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]

df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

cols = list(df_pct_chg_t.columns)
cols = [cols[-1]] + cols[:-1]
df_pct_chg_t = df_pct_chg_t[titles_bs]


df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index
df_t.index = range(len(df_t))

cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
#endregion

df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')

df_t = df_t[df_t.columns[::-1]]
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]
df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())
df_n_sum[df_n_sum == 0] = ""
new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum[1:] #take the data less the header row
df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])

# return render_template('current_ratio.html', company_symbol = profiles_dict['symbol'],\
#                         company_long_name = profiles_dict['long name'],\
#                         company_currency = profiles_dict['currency'],\
#                         company_exchange = profiles_dict['exchange'],\
#                         company_industry = profiles_dict['industry'],\
#                         company_description = profiles_dict['description'],\
#                         company_sector = profiles_dict['sector'],\
#                         company_country = profiles_dict['country'],\
#                         company_ipo_date = profiles_dict['ipo date'],\
#                         company_short_name = profiles_dict['short name'],\
#                         first_num = first_num,\
#                         latest_num = latest_num,\
#                         historical_pct_chg = historical_pct_chg,\
#                         lifetime_sum_all_metric = lifetime_sum_all_metric,\
#                         mean_str = mean_str,\
#                         max_str = max_str,\
#                         min_str = min_str,\
#                         std_dev_str = std_dev_str,\
#                         bottom_25_str = bottom_25_str,\
#                         top_25_str = top_25_str,\
#                         earliest_year = earliest_year,\
#                         latest_year = latest_year,\
#                         earliest_metric = earliest_metric,\
#                         latest_metric = latest_metric,\
#                         max_min_pct_diff_str = max_min_pct_diff_str, df_bs_table_html = [df_table_html],df_html_tall = [df_html_tall],fin_metric_name = fin_metric_title,\
#                         df_date = df['date'].to_list(), df_rev = df["{}".format(fin_metric_name)].to_list(),\
#                         df_json  =df.to_numpy().tolist(),\
#                         table_pct = [df_pct],\
#                         tables=[df_html],\
#                         titles=df.columns.values,\
#                         total_time=total_seconds,\
#                         place_name=url_symbol,\
#                         max=17000,\
#                         labels=labels,\
#                         values=values)


KeyError: "None of [Index(['Cash and Cash Equivalents', 'Short Term Investments',\n       'Cash And Short-Term Investments', 'Accounts Receivables', 'Inventory',\n       'Other Current Assets', 'Total Current Assets', 'PP&E', 'Goodwill',\n       'Intangible Assets', 'Goodwill and Intangible Assets',\n       'Long-term Investments', 'Tax Assets', 'Other Non-Current Assets',\n       'Total Non-Current Assets', 'Other Assets', 'Total Assets',\n       'Accounts Payables', 'Short-term Debt', 'Income Tax Payables',\n       'Deferred Revenue', 'Other Current Liabilities',\n       'Total Current Liabilities', 'Long-term Debt',\n       'Deferred Revenue Non-Current', 'Deferred Tax Liabilities Non-Current',\n       'Other Non-Current Liabilities', 'Total Non-Current Liabilities',\n       'Other Liabilities', 'Total Liabilities', 'Common Stock',\n       'Retained Earnings', 'Accumulated Other Comprehensive Income Loss',\n       'Other Shareholders Equity', 'Total Shareholders Equity',\n       'To

In [1]:








# df_fin_statement = df_fin_statement[cols]

df_fin_statement.index = df_fin_statement['Quarter & Year']

million = 1000000
billion = 1000000000
fin_metric_history = df_fin_statement['{}'.format(fin_metric_title)]
if list(fin_metric_history)[0]  > billion:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/billion).round(decimals=2)


elif list(fin_metric_history)[0]  > million:
    df_fin_statement['{}'.format(fin_metric_title)] = (fin_metric_history/million).round(decimals=2)

    pass
else:
    pass

df_pct_chg = df_fin_statement
pct_chg_cols = (df_fin_statement.select_dtypes(include=['number']).pct_change(-1))
df_pct_chg_str = df_pct_chg.drop(list(pct_chg_cols), axis=1)
df_pct_chg = df_pct_chg_str.join(pct_chg_cols)[list(df_fin_statement)]

pd.set_option('display.float_format', '{:.2f}'.format)
df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]

df_pct_chg_t = df_pct_chg.transpose()
df_pct_chg_t.columns = list(df_pct_chg['Quarter & Year'])
df_pct_chg_t = df_pct_chg_t.iloc[1:]
df_pct_chg_t['']=df_pct_chg_t.index
df_pct_chg_t.index = range(len(df_pct_chg_t))

cols = list(df_pct_chg_t.columns)
cols = [cols[-1]] + cols[:-1]
df_pct_chg_t = df_pct_chg_t[titles_bs]


df_t = df_fin_statement.transpose()
df_t.columns = list(df_fin_statement['Quarter & Year'])
df_t = df_t.iloc[1:]
df_t['']=df_t.index
df_t.index = range(len(df_t))

cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]
#endregion

df_pct = df_pct_chg_t.to_html().replace('<table','<table class="df_tableBoot" id="df_myTable1"')# dt-responsive" id="df_myTable"')

df_t = df_t[df_t.columns[::-1]]
cols = list(df_t.columns)
cols = [cols[-1]] + cols[:-1]
df_t = df_t[cols]


def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False
df_n = df_fin_statement[df_fin_statement.applymap(isnumber)]
df_n[df_n < 2] = np.nan
df_n_sum = pd.DataFrame(df_n.sum())
df_n_sum[df_n_sum == 0] = ""
new_header = df_n_sum.iloc[0] #grab the first row for the header
df_n_sum = df_n_sum[1:] #take the data less the header row
df_n_sum.columns = new_header #set the header row as the df header
df_n_sum.index = range(len(df_n_sum))
df_t = pd.merge(df_n_sum, df_t, left_index=True, right_index=True,suffixes=('Total: {} - {}'.format(latest_year,earliest_year), 'Line Items'))
df_t = df_t[0:25]



col_list = []
n=0
while n<len(list(df_t))-0:
    if n<6:
            col_item = '<col id="col_item_{}" class="col_item_class first_7_col">'.format(n)
    else:
        col_item = '<col id="col_item_{}" class="col_item_class">'.format(n)
    col_list.append(col_item)
    n+=1
col_list_str = ''.join(map(str, col_list))
df_html = df_t.to_html().replace('border="1" class="dataframe">','class="df_tableBoot" id="df_myTable" border="1" class="dataframe"><colgroup>{}</colgroup>'.format(col_list_str))
# df_table_html = df_fin_statement[['{}'.format(fin_metric_title)]].iloc[::-1].transpose().to_html()#.replace("\n","")
df_tall = df.iloc[::-1]
# df_tall.index = df['Quarter & Year']
# df_tall = df.index.shift(-1)
# df_tall = df_tall.reset_index()
df_html_tall = df_tall[['{}'.format('Quarter & Year'),'{}'.format(fin_metric_name)]].to_html(index=False)
df_html_tall = df_html_tall.replace('border="1" class="dataframe">','class="abc" id="df_myTable" border="1" class="dataframe">')#.replace("'","")
# df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("\n","")
df_html_tall = df_html_tall.replace("{}".format("["),"")
df_html_tall = df_html_tall #render_template_string(df_html_tall)
df_html = df_html.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html = df_html.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
#  df_html_tall = df_html_tall.replace("\n","")
# df_html_tall.replace("\n",'">')
df_html_tall = df_html_tall.replace('<td>','<td class="td_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<th>','<th class="th_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall.replace('<tr>','<tr class="tr_fin_statement_class fin_statement_class">')
df_html_tall = df_html_tall[0:]

df = df[['date',"{}".format(fin_metric_name)]].dropna() #.fillna(0)#.fillna(method='bfill')
df['date'] = pd.to_datetime(df['date']).values.astype(np.int64) // 10 ** 6
full_path = csv_file.split(' ~ ')
path = pathlib.PurePath(full_path[0])
print("Nothing took {} seconds".format(time.time() - start_time))
total_seconds = ((time.time() - start_time))

labels = [
    'JAN', 'FEB', 'MAR', 'APR',
    'MAY', 'JUN', 'JUL', 'AUG',
    'SEP', 'OCT', 'NOV', 'DEC'
]
labels = list(df['date'])#[0:19]

values = [
    967.67, 1190.89, 1079.75, 1349.19,
    2328.91, 2504.28, 2873.83, 4764.87,
    4349.29, 6458.30, 9907, 16297
]

values = list(df["{}".format(fin_metric_name)])#[0:19]

colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]
df_table_html = df_tall[['{}'.format(fin_metric_name)]].iloc[::-1].transpose().to_html()#.replace("\n","")
present_num = magnitude_num(int(latest_metric),currency_symbol)
latest_num = "${}".format(df["{}".format(fin_metric_name)][0])

# return render_template('current_ratio.html', company_symbol = profiles_dict['symbol'],\
#                         company_long_name = profiles_dict['long name'],\
#                         company_currency = profiles_dict['currency'],\
#                         company_exchange = profiles_dict['exchange'],\
#                         company_industry = profiles_dict['industry'],\
#                         company_description = profiles_dict['description'],\
#                         company_sector = profiles_dict['sector'],\
#                         company_country = profiles_dict['country'],\
#                         company_ipo_date = profiles_dict['ipo date'],\
#                         company_short_name = profiles_dict['short name'],\
#                         first_num = first_num,\
#                         latest_num = latest_num,\
#                         historical_pct_chg = historical_pct_chg,\
#                         lifetime_sum_all_metric = lifetime_sum_all_metric,\
#                         mean_str = mean_str,\
#                         max_str = max_str,\
#                         min_str = min_str,\
#                         std_dev_str = std_dev_str,\
#                         bottom_25_str = bottom_25_str,\
#                         top_25_str = top_25_str,\
#                         earliest_year = earliest_year,\
#                         latest_year = latest_year,\
#                         earliest_metric = earliest_metric,\
#                         latest_metric = latest_metric,\
#                         max_min_pct_diff_str = max_min_pct_diff_str, df_bs_table_html = [df_table_html],df_html_tall = [df_html_tall],fin_metric_name = fin_metric_title,\
#                         df_date = df['date'].to_list(), df_rev = df["{}".format(fin_metric_name)].to_list(),\
#                         df_json  =df.to_numpy().tolist(),\
#                         table_pct = [df_pct],\
#                         tables=[df_html],\
#                         titles=df.columns.values,\
#                         total_time=total_seconds,\
#                         place_name=url_symbol,\
#                         max=17000,\
#                         labels=labels,\
#                         values=values)


NameError: name 'df_fin_statement' is not defined

In [19]:
len(cols)

28